In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE77627"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE77627"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE77627.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE77627.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE77627.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Whole genome profiling of liver biopsies discloses potential biomarkers for diagnosis of idiopathic non-cirrhotic portal hypertension"
!Series_summary	"Background. Idiopathic non-cirrhotic portal hypertension (INCPH) is a frequently misdiagnosed cause of portal hypertension. It also lacks a specific test for its diagnosis. This study evaluates whether using new immunohistochemistry makers derived from whole genome analysis improves the diagnosis of INCPH.  Methods. We analyzed formalin-fixed, paraffin embedded (FFPE) liver tissue from 18 INCPH and 22 patients with cirrhosis (LC) as well as from 14 histologically normal livers (HNL) as controls. Microarray experiments were performed using Illumina Whole-Genome DASL HT BeadChip arrays. Selected genes showing differential expression at Illumina were confirmed using quantitative real-time PCR (qRT-PCR) gene expression performed with Fluidigm Biomark HD system in a subgroup of samples. Immunohistochemi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine gene expression data availability
# Based on the background information, this dataset contains transcriptomic data from liver tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows for trait, age, and gender

# For trait (Hypertension):
# In this study, the dataset contains information about liver groups: HNL (normal), INCPH (idiopathic non-cirrhotic portal hypertension), and LC (cirrhosis)
# Row 0 contains the liver group classification which can be used to identify hypertension status
trait_row = 0  # 'liver group' is in row 0

# Age and gender are not provided in the sample characteristics dictionary
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert liver group values to binary hypertension status.
    INCPH (idiopathic non-cirrhotic portal hypertension) -> 1 (has hypertension)
    LC (liver cirrhosis) and HNL (histologically normal livers) -> 0 (no hypertension)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        parts = value.split(":", 1)
        value = parts[1].strip()
    
    # Convert to binary
    if "INCPH" in value:
        return 1  # INCPH is a type of portal hypertension
    elif "HNL" in value or "LC" in value:
        return 0  # Normal livers or cirrhosis without explicit portal hypertension
    else:
        return None

def convert_age(value):
    """
    Placeholder function for age conversion (not used as age data is unavailable)
    """
    return None

def convert_gender(value):
    """
    Placeholder function for gender conversion (not used as gender data is unavailable)
    """
    return None

# 3. Save Metadata - Conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical_data that was prepared in a previous step
    # Do not create a new DataFrame, we should be using the one from the previous step
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # clinical_data should exist from a previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the DataFrame
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data is not available from previous steps.")


Preview of selected clinical features:
{'GSM2055372': [0.0], 'GSM2055373': [0.0], 'GSM2055374': [0.0], 'GSM2055375': [0.0], 'GSM2055376': [0.0], 'GSM2055377': [0.0], 'GSM2055378': [0.0], 'GSM2055379': [0.0], 'GSM2055380': [0.0], 'GSM2055381': [0.0], 'GSM2055382': [0.0], 'GSM2055383': [0.0], 'GSM2055384': [0.0], 'GSM2055385': [0.0], 'GSM2055386': [1.0], 'GSM2055387': [1.0], 'GSM2055388': [1.0], 'GSM2055389': [1.0], 'GSM2055390': [1.0], 'GSM2055391': [1.0], 'GSM2055392': [1.0], 'GSM2055393': [1.0], 'GSM2055394': [1.0], 'GSM2055395': [1.0], 'GSM2055396': [1.0], 'GSM2055397': [1.0], 'GSM2055398': [1.0], 'GSM2055399': [1.0], 'GSM2055400': [1.0], 'GSM2055401': [1.0], 'GSM2055402': [1.0], 'GSM2055403': [1.0], 'GSM2055404': [0.0], 'GSM2055405': [0.0], 'GSM2055406': [0.0], 'GSM2055407': [0.0], 'GSM2055408': [0.0], 'GSM2055409': [0.0], 'GSM2055410': [0.0], 'GSM2055411': [0.0], 'GSM2055412': [0.0], 'GSM2055413': [0.0], 'GSM2055414': [0.0], 'GSM2055415': [0.0], 'GSM2055416': [0.0], 'GSM2055417': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina probe IDs, not human gene symbols
# They start with "ILMN_" which is a standard prefix for Illumina microarray probes
# These need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Addres

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation data that store gene identifiers and gene symbols
probe_id_column = "ID"  # Column containing the same identifiers as in gene_data
gene_symbol_column = "Symbol"  # Column containing the gene symbols

# 2. Get the gene mapping dataframe using the two columns
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=probe_id_column, gene_col=gene_symbol_column)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Print the first few rows of the mapped gene expression data
print("Preview of mapped gene expression data:")
print(gene_data.index[:20])


Preview of mapped gene expression data:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC', 'AADACL1',
       'AADACL2', 'AADACL4', 'AADAT', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data instead of re-extracting it
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Clinical data loaded from {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# If the clinical data has unnamed index column (typical when saving without index=False),
# let's set the first column as the index
if clinical_df.columns[0] == 'Unnamed: 0':
    clinical_df = clinical_df.set_index(clinical_df.columns[0])

# Ensure the trait column exists by transposing the clinical data
# In our case, it was stored with samples as columns, traits as rows
clinical_df = clinical_df.T
clinical_df.columns = [trait]  # Rename the column to match our trait variable
print("Transposed clinical data preview:")
print(preview_df(clinical_df))

# 3. Link the clinical and genetic data
linked_data = pd.merge(clinical_df, normalized_gene_data.T, left_index=True, right_index=True, how='inner')
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns preview:")
print(linked_data.columns[:10])  # Show first 10 columns to verify structure

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Verify that the trait column has at least two unique values
unique_trait_values = linked_data[trait].unique()
print(f"Unique values in trait column: {unique_trait_values}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains liver tissue samples with information about portal hypertension status."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Hypertension/gene_data/GSE77627.csv
Clinical data loaded from ../../output/preprocess/Hypertension/clinical_data/GSE77627.csv
Clinical data shape: (1, 54)
Clinical data preview:
{'GSM2055372': [0.0], 'GSM2055373': [0.0], 'GSM2055374': [0.0], 'GSM2055375': [0.0], 'GSM2055376': [0.0], 'GSM2055377': [0.0], 'GSM2055378': [0.0], 'GSM2055379': [0.0], 'GSM2055380': [0.0], 'GSM2055381': [0.0], 'GSM2055382': [0.0], 'GSM2055383': [0.0], 'GSM2055384': [0.0], 'GSM2055385': [0.0], 'GSM2055386': [1.0], 'GSM2055387': [1.0], 'GSM2055388': [1.0], 'GSM2055389': [1.0], 'GSM2055390': [1.0], 'GSM2055391': [1.0], 'GSM2055392': [1.0], 'GSM2055393': [1.0], 'GSM2055394': [1.0], 'GSM2055395': [1.0], 'GSM2055396': [1.0], 'GSM2055397': [1.0], 'GSM2055398': [1.0], 'GSM2055399': [1.0], 'GSM2055400': [1.0], 'GSM2055401': [1.0], 'GSM2055402': [1.0], 'GSM2055403': [1.0], 'GSM2055404': [0.0], 'GSM2055405': [0.0], 'GSM2055406': [0.0], 'GSM2055407': [0.0], 'GSM2055408

Data shape after handling missing values: (54, 19451)
Unique values in trait column: [0. 1.]
For the feature 'Hypertension', the least common label is '1.0' with 18 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Hypertension' in this dataset is fine.



Processed dataset saved to ../../output/preprocess/Hypertension/GSE77627.csv
